# ERP 원본 데이터
- 제품군-GITEM-SITEM: GITEM 정보
- GITEM_ 공정-순서 (공정군 분류 정보 추가)
- 수율(공정사용 기준 미정)
- 라인스피드 (확정)
- P/O 정보: 기준 X, 현재로서는 이전에 있던 정보 그대로 사용

# 생산계획 원본 데이ㅌ터
- 폭변경: 수정없이 유지
- 공정교체시간: 호기코드로 변경 및 호기코드 컬럼명 수정

# 사용자 입력
- 기준날짜 (base_date)
- 윈도우 크기 (window_size)
- 사용하는 수율/라인스피드 데이터 기준 (yield_period, linespeed_period)

In [3]:
import pandas as pd

gitem_info = pd.read_excel("샘플데이터.xlsx", sheet_name = "제품군-GITEM-SITEM", skiprows = 2)
gitem_info = gitem_info.drop(columns = {"영문명"})

gitem_operation_sequence = pd.read_excel("샘플데이터.xlsx", sheet_name = "GITEM-공정-순서", skiprows = 1)
gitem_operation_sequence = gitem_operation_sequence.drop(columns = {"영문명"})

linespeed = pd.read_excel("샘플데이터.xlsx", sheet_name = "라인스피드-GITEM등", skiprows = 5)
linespeed = linespeed.drop(columns = {"영문명"})

In [6]:
# PYTHON_INPUT: 3. OPERATION_TYPES

operation_type = gitem_operation_sequence[['PROCCODE', 'PROCNAME','ProcGbn']].drop_duplicates()
operation_type

,PROCCODE,PROCNAME,ProcGbn
0,20300,염료점착,염료
1,20700,유광 S/R,SR
2,20902,안료접착,안료
3,20706,MIBK SR,SR
4,20500,투명점착,투명
5,20906,인쇄,인쇄
7,20706,MIBK SR,SR
14,23311,PPF점착,투명
15,12400,TOP COATING,SR


In [13]:
# PYTHON_INPUT: 4. GITEM_OPERATION
gitem_operation = gitem_operation_sequence.pivot(
    index = ["GitemNo"],
    values = ["PROCCODE"],
    columns = ["PROCSEQ"]
).reset_index()
gitem_operation

GitemNo PROCCODE                           
PROCSEQ                1        2        3        4
0         31540  20300.0  20700.0      NaN      NaN
1         31704  23311.0  12400.0      NaN      NaN
2         31798  20902.0  20706.0  20500.0  20906.0
3         31800  20902.0  20706.0  20500.0  20906.0
4         31802  20902.0  20706.0  20500.0  20906.0
5         31864  20300.0  20700.0      NaN      NaN

In [9]:
# PYTHON_INPUT: 2. LINESPEED

# 컬럼명 구조 설명
# L: Linespeed
# 첫번째 숫자: gitem 분류, gitem, 4분류, 3분류, 2분류중 무엇인지
# 두번째 숫자: 1년, 6개월, 3개월중 뭘 사용하는지

# 1. 1년, 6개월, 3개월 중 사용자에게 입력받은 값대로 필요한 데이터 입력받고 해당 컬럼만 남김
# 2. 그중 null이 아닌 첫번째 숫자만 남김. 이떄 컬럼명 ['operation_linespeed'], 이때 선택된 컬럼을 ['standard']
# 3.



import re

def linespeed_selection(linespeed, linespeed_period):

    if linespeed_period == '6_months':
        linespeed_cols = [col for col in linespeed.columns if re.match(r"^L.*1$", col)] # ㅣ로 시작, 1로 끝나는 컬럼
        print(f"6개월 기준 컬럼명{linespeed_cols}") 
    elif linespeed_period == '1_year':
        linespeed_cols = [col for col in linespeed.columns if re.match(r"^L.*2$", col)] # L로 시작, 2로 끝나는 컬럼
        print(f"1년 기준 컬럼명{linespeed_cols}") 
    elif linespeed_cols == '3_months':
        linespeed_cols = [col for col in linespeed.columns if re.match(r"^L.*2$", col)] # L로 시작, 2로 끝나는 컬럼
        print(f"3개월 기준 컬럼명{linespeed_cols}") 
    # 첫번째로 NaN이 아닌 값으로 Linespeed 채우기
    linespeed['linespeed'] = linespeed[linespeed_cols].bfill(axis=1).iloc[:, 0] 

    # linespeed로 선택된 컬럼명 가져오기
    linespeed['selected'] = linespeed[linespeed_cols].apply(lambda x: x.first_valid_index(), axis=1)
    
    # 사용하지 않는 컬럼(L로 시작하는 컬럼) 제외
    linespeed = linespeed.drop(columns = linespeed.columns[linespeed.columns.str.startswith("L")])
    return linespeed
    
    
selected_linespeed = linespeed_selection(linespeed, linespeed_period = '6_months')

linespeed_pivot = selected_linespeed.pivot(
    index = ['GitemNo','PROCCODE'],
    columns = 'MachineNo',
    values = 'linespeed'
).reset_index().rename_axis(None, axis=1)

linespeed_pivot

6개월 기준 컬럼명['L11', 'L21', 'L31', 'L41', 'L51']


,GitemNo,PROCCODE,C2010,C2210,C2250,C2260,C2270
0,31540,20300,NaN,NaN,NaN,20.0,NaN
1,31540,20700,NaN,NaN,NaN,NaN,40.0
2,31704,12400,10.0,NaN,NaN,NaN,NaN
3,31704,23311,10.0,NaN,NaN,NaN,NaN
4,31798,20500,NaN,NaN,NaN,26.4,26.4
5,31798,20706,8.3,NaN,NaN,NaN,NaN
6,31798,20902,NaN,NaN,NaN,18.0,NaN
7,31798,20906,18.0,NaN,17.3,NaN,NaN
8,31800,20500,NaN,NaN,NaN,NaN,27.0
9,31800,20706,NaN,NaN,NaN,NaN,40.0


In [26]:
machine_master_info = (
    linespeed[['MachineNo', 'MachineName']]
    .drop_duplicates()
    .sort_values(by = 'MachineNo')
    .reset_index(drop=True)
    .assign(MachineIndex=lambda df: range(len(df)))
    )
machine_master_info

,MachineNo,MachineName,MachineIndex
0,C2010,NaN,0
1,C2210,NaN,1
2,C2250,NaN,2
3,C2260,NaN,3
4,C2270,NaN,4
